![logo_dymaxion](img/dymaxion_labs.png)

# Clasificación de imágenes satelitales con CNNs usando Keras

Vamos a presentar una posible aplicación de redes convolucionales (CNN) para imágenes satelitales. En este caso, buscaremos detectar asentamientos precarios usando solamente esa fuente de datos. De esta manera, podemos actualizar periódicamente la cantindad de km2 de asentamientos y así tener un indicador que pueda anticiparse a las estadísticas de hábitat.

Luego vamos a mostrar un ejemplo usando segmentación.

Para más información pueden ver nuestro sitio [www.dymaxionlabs.com](http://www.dymaxionlabs.com).

## Mini introducción a imágenes satelitales

Existen distintos tipos de imágenes satelitales (ópticas), una clasificación que se puede hacer es de acuerdo a su resolución espacial.

Podemos distinguir entre (no exhaustivo):

* **Resolución baja**: [MODIS](https://modis.gsfc.nasa.gov/), 250-1000m por pixel.
    * Utilizadas para investigación en cambio climático, recursos naturales y análisis de grandes superficies.
    * Imagen diaria.
    * Gratuitas, disponibles en varias fuentes. Por ejemplo [Google Earth Engine](https://earthengine.google.com/)
    * Casos de uso: predicción de cosecha para el agro para grandes supericies. Indices de calidad del aire.


* **Resolución media**: Landsat (30m), Sentinel-2 (10m), RapidEye (5m).
    * Utilizadas para análisis urbano y agro.
    * Cada 15 días (Landsat) y 5 días (Sentinel-2).
    * Landsat y Sentinel-2 son gratuitas, disponibles en varias fuentes.
    * Casos de uso: predicción de cosecha e indicadores relacionados con agricultura de precisión.


* **Resolución alta**: WorldView-4 (0.3m de resolución)
    * utilizadas para detección de objetos y monitoreo de gran precisión.
    * Revisita variable según la zona.
    * No son gratuitas (~24 USD/km2).
    * Casos de uso: conteo de autos en parking lots de supermercados y contenedores en puertos.

### Ejemplo Museo Louvre (Paris, Francia)

#### Sentinel-2

![louvre_sen](img/louvre_sen.png)

#### WorldView-4

![louvre_wv](img/louvre_wv.jpg)

### Redes Neuronales y Deep Learning

Tradicionalmente se utilizaron métodos cómo clasificación por pixel o Análisis de Imágenes basada en objectos (OBIA). Un enfoque que que se hizo popular recientemente es el de usar redes neuronales convolucionales para hacer la clasificación de imágenes. Se combina con técnicas de *computer vision* para filtrar y segmentar las imágenes de acuerdo a la clase que se busca detectar.

A continuación va un ejemplo de cómo procesar imáges de un territorio para clasificar asentamientos precarios usando redes neuronales convolucionales. Vamos a hacer transfer learning, aprovechando de redes ya entrenadas.

## Preprocesamiento

Las imágenes tal cual son capturadas por los sensores de los satélites deben ser preprocesadas para que puedan ser utilizados. El proceso tiene varias etapas:

* **Corrección radiométrica**
* **Ortorrectificación**
* **Corrección atmosférica**

No vamos a entrar en detalle acá porque llevaría mucho tiempo y se va del scope de esta charla :)

## Construcción del dataset

Se particiona la imagen entera en tiles de `n`x`m` pixeles y se cruza con las localizaciones de asentamientos precarios para establecer cuales son True y cuales False. Las dimensiones van de acuerdo a la red que usemos.

#### Ejemplos de True

![guatemala_vya_t](img/guatemala_vya_t.jpg) ![guatemala_vya_t_2](img/guatemala_vya_t_2.jpg)

#### Ejemplos de False

![guatemala_no_vya_f](img/guatemala_no_vya_f.jpg) ![guatemala_no_vya_f_2](img/guatemala_no_vya_f_2.jpg)

Dada la estructura de los directorios,

```
data/
  train/
    true/
    false/
  validation/
    true/
    false/
```

levantamos los datos con una estructura que permita trabajar con Keras.

In [4]:
from glob import glob
import os

In [5]:
train_files = glob(os.path.join('data/', 'train/**/', '*.jpg'))
validation_files = glob(os.path.join('data/', 'validation/**/', '*.jpg'))

img_width, img_height = 256, 256

train_data_dir = "data/train/"
validation_data_dir = "data/validation/"

nb_train_samples = len(train_files)
nb_validation_samples = len(validation_files)

batch_size = 64
epochs = 200

#### Inicializamos una red ResNet50

In [8]:
from keras.applications.resnet50 import ResNet50

![paper_resnet](img/paper_resnet.png)

In [9]:
model = ResNet50(
    weights = "imagenet",
    include_top = False,
    input_shape = (img_width, img_height, 3))

94633984/94653016 [============================>.] - ETA: 0ss8

Si bien la arquitectura de esta red es más profunda que el caso de VGG16 y VGG19, el hecho de que use _global average pooling_ en lugar de capas totalmente conectadas hace que su peso sea menor.

#### Freezamos las primeras 60 capas, reentrenamos las restantes

In [12]:
for layer in model.layers[:60]:
    layer.trainable = False

#### Agregamos una última capa al final para obtener las predicciones

In [13]:
from keras.layers import Flatten, Dense, Dropout

In [14]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation = "sigmoid")(x)

#### Cerramos el modelo y lo compilamos

In [17]:
from keras.models import Model
from keras import optimizers

In [18]:
model_final = Model(inputs = model.input, outputs = predictions)

model_final.compile(
    loss = 'binary_crossentropy',
    optimizer = optimizers.SGD(lr = 0.0001, momentum = 0.9),
    metrics = ['accuracy'])

#### Generamos los conjuntos de datos de entrenamiento y validación

Usamos data agumentation para contemplar rotaciones y otras variaciones. El concepto aquí es que no nos importa en qué parte de la imagen está la clase que queremos detectar.

In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    rotation_range = 30)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    rotation_range = 30)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "binary")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    class_mode = "binary")

Finalmente, usamos *early stopping* y usamos checkpoints para ir guardando cada corrida. Al final guardamos los pesos de la red.

In [22]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("./models/demo_weights.hdf5",
    monitor = 'val_acc',
    verbose = 1,
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto',
    period = 1)

early = EarlyStopping(
    monitor = 'val_acc',
    min_delta = 0,
    patience = 10,
    verbose = 1,
    mode = 'auto')

model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs, 
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = [checkpoint, early])

model_final.save('./models/demo.h5')
print('Done with {} layers')

## Post-procesamiento

Luego de esto hay que proceder a filtrar los resultados y segmentar las imágenes para terminar de detectar los objetos relevantes.

Para segmentar las imágenes se puede usar la librería scikit-image u OpenCV.

Algo relativamente sencillo que se puede hacer en este caso es:
1. Remover los rectángulos pequeños y de baja probabilidad con un filtro de mediana
2. Fusionar rectángulos que se solapan en un sólo polígono

## Resultado

#### ([link al mapa](http://dymaxionlabs.com/ap-latam/en/map/?id=guatemala))

![Guatemala](img/guatemala_map.jpg)

![Mar del Plata](img/mdq_map.png)

Esto lo hacemos con máquinas de [Google Cloud Platform](https://cloud.google.com). También usamos [Floydhub](https://floydhub.com) para las corridas de las redes neuronales.

## Otros casos de uso

Se suelen usar redes neuronales profundas también para poder clasificar usos del suelo. A partir de esto se pueden crear mapas a partir de imágenes como también detectar cambios a lo largo del tiempo para una región dada.

Para esto se utilizan técnicas como las que comentamos anteriormente como sliding windows o filtros. Sin embargo, existen arquitecturas de redes que se pueden usar para realizar esa segmentación. Un ejemplo de esto es la red [U-Net](https://blog.deepsense.ai/deep-learning-for-satellite-imagery-via-image-segmentation/).

Este tipo de red fue creada para trabajar con imágenes médicas pero también se usa para otros contextos.

La arquitectura U-Net está implementada en Keras. Se puede acceder en https://github.com/zhixuhao/unet.

![U-net paper](img/u_net_paper.png)

![U-Net](img/u_net_example.png)

![U-Net img2map](img/img2map.png)

En Dymaxion Labs usamos estas técnicas para detectar cambios a lo largo del tiempo. Tenemos un cliente con el que trabajamos esto para detectar parcelas sin permiso de obra.

![Change 1](img/seg_change1.png)

![Change 2](img/seg_change2.png)

![meme](img/meme_ortiba.jpg)